<a href="https://colab.research.google.com/github/Graur/mipt/blob/master/NLP_home_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ №1, Граур Андрей Константинович, курс NLP

Вам необходимо решить задачу нахождения похожих товаров в интернет-магазинах по их описанию.

1. Добавим необходимые мне импорты

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

2. Скачиваем наш датасет https://www.kaggle.com/datasets/cclark/product-item-data/data

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample-data.csv')

3. Проверяем данные

In [8]:
data.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


4. Создадим объекты для векторизации текста с использованием TF-IDF и Word2Vec

In [9]:
tfidf_vectorizer = TfidfVectorizer()
word2vec_model = Word2Vec(data['description'].apply(lambda x: x.split()), min_count=1)

5. Преобразуем текстовые описания товаров в векторное представление с помощью TF-IDF


In [10]:
tfidf_vectors = tfidf_vectorizer.fit_transform(data['description'])

6. Аналогично, преобразуем текстовые описания товаров в векторное представление с помощью Word2Vec

In [11]:
word2vec_vectors = data['description'].apply(lambda x: word2vec_model.wv[x.split()].mean(axis=0))

7. Вычислим косинусное расстояние между всеми парами векторов:


In [13]:
cosine_distances_tfidf = cosine_distances(tfidf_vectors)

In [17]:
# Преобразование word2vec_vectors в двумерный массив
word2vec_vectors = np.array([np.array(v) for v in word2vec_vectors])

# Рассчет косинусных расстояний
cosine_distances_word2vec = cosine_distances(word2vec_vectors)

8. Установим порог значимости и определим, похожи ли два текста или нет

In [32]:
threshold = 0.5

similar_products_tfidf = []
similar_products_word2vec = []

for i in range(len(data)):
    for j in range(i+1, len(data)):
        if cosine_distances_tfidf[i][j] < threshold:
            similar_products_tfidf.append((data['description'][i], data['description'][j]))
        if cosine_distances_word2vec[i][j] < threshold:
            similar_products_word2vec.append((data['description'][i], data['description'][j]))

In [33]:
similar_products_tfidf[0]

('Active sport boxer briefs - Skinning up Glory requires enough movement without your boxers deciding to poach their own route. The form-fitting Active Sport Boxer Briefs are made from breathable 93% polyester (71% recycled) fabric that\'s fast-wicking, dries quickly and has 7% spandex for stretch; the seamless waistband and soft leg edges won\'t roll or bind. The gusseted, flat-sewn 6" inseam (size M) is offset to prevent inner-thigh chafe. Fly-free with a smooth front panel. Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Stretch mesh provides support, open-weave mesh for airflow, wicks efficiently, dries fast"</li> <li>Seamless construction</li> <li>"Flat-sewn, gusseted inseam is set forward to prevent inner-thigh chafe"</li> <li>Fly-free support</li> <li>"Inseam (size M) is 6"""</li></ul><br><br><b>Fabric: </b>"4.6-oz 93% polyester (71% recycled)/7% spandex, with moisture-wicking performance. Recyclable through the Common Threads Recycling P

In [34]:
similar_products_word2vec[0]

('Active classic boxers - There\'s a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>Fabric: </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><

In [35]:
len(similar_products_tfidf)

872

In [36]:
len(similar_products_word2vec)

124750